<a href="https://colab.research.google.com/github/AmbarishChandurkar/Analog-Digital-Communication-MATLABFileexchange-/blob/master/BG_ExtremeCase_30Files_CorrectedFeatures(older).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from importlib import reload
reload(iup)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Part 1 : Data Import**

## **Step 1.1 : Get the Utterance Files** 

In [ ]:
!pip install tftb

In [ ]:
import sys
sys.path.append('/content/IEMOCAPFeatures_Utility.py')
import IEMOCAPFeatures_Utility as iup

In [ ]:
import IEMOCAPFeatures_Utility as iup
import pandas as pd
Sessions = [1,2,3,4,5]
Utterances = pd.DataFrame()
for S in Sessions:
  Utterances = pd.concat([Utterances,pd.read_pickle('/content/drive/My Drive/IIIT  College files/4th Year - 7th Semester (Internship)/IEMOCAP_DB/Features and Pickles/UtteranceInfo_S' + str(S) + '.pkl')])  
Utterances.head()

In [ ]:
iup.GetSummary(Utterances)

In [ ]:
NT = 2.4
PT = 3.9
Labels = {'Negative':0,'Neutral':1,'Positive':2}
Utterances['V'] = Utterances['V'].apply(lambda x : Labels['Negative'] if x < NT \
                                        else Labels['Positive'] if x > PT else Labels['Neutral'])

In [ ]:
Temp = pd.DataFrame()
for V in Labels.values():
  Temp = pd.concat([Temp, FeatureSet[FeatureSet['V'] == V].sample(int(min(iup.GetSummary(FeatureSet)['Count'])))],ignore_index=True)
FeatureSet = Temp

In [ ]:
from sklearn.model_selection import train_test_split
Labels, nolabels = train_test_split(Utterances,stratify = Utterances.V,test_size = 0.55)

In [ ]:
iup.GetSummary(Labels)

## **Step 1.2 : Import Feature Data**

In [ ]:
!unzip -q "/content/drive/My Drive/IIIT  College files/4th Year - 7th Semester (Internship)/IEMOCAP_DB/Session1.zip" -d "content/"

In [ ]:
FeatureTable = iup.GetTDD_FeatureTable(Labels,'/content/content/content/IEMOCAP_full_release/',\
                                      int(Fs*Shift*1e-3),int(Fs*WS*1e-3),TDDShape)
'''
FeatureTable = pd.DataFrame()
for S in Sessions:
  FeatureTable = pd.concat([FeatureTable,pd.read_pickle('/content/drive/My Drive/IIIT  College files/4th Year - 7th Semester (Internship)/IEMOCAP_DB/Features and Pickles/TDDF_S' + str(S) + '_new.pkl')])
FeatureTable.head()
'''

In [ ]:
FeatureTable.head()

In [ ]:
FeatureTable.Feature.iloc[0].shape

## **Step 1.2 : Import Features**

In [ ]:
import pandas as pd
TDDF_S1 = pd.read_pickle('/media/ambarish/New Volume/IEMOCAP_DB/Data/Features/TDDF_S1_new.pkl')
FeatureTable = TDDF_S1
FeatureTable

## **Step 2.1 : Select Files and corresponding frames for TrainData**

In [ ]:
Files_First50 = Files['TURN_NAME'].iloc[0:50]
Files_First50

In [ ]:
from tqdm import tqdm
TrainData = pd.DataFrame([])
for Filename in tqdm(Files_First50):
    TempTable = FeatureTable
    Temp = TempTable.loc[TempTable['SampleName'].apply(lambda x : x.startswith(Filename))]
    TrainData = pd.concat([TrainData,Temp])

In [ ]:
TrainData

## **Step 2.2 : Select Files and corresponding frames for TestData**

In [ ]:
Files_Next50 = Files['TURN_NAME'].iloc[50:100]
Files_Next50

In [ ]:
from tqdm import tqdm
DevData = pd.DataFrame([])
for Filename in tqdm(Files_Next50):
    TempTable = FeatureTable
    Temp = TempTable.loc[TempTable['SampleName'].apply(lambda x : x.startswith(Filename))]
    DevData = pd.concat([DevData,Temp])

In [ ]:
DevData

## **Step 2.3 : Get Summary for Train and Test Data**

In [ ]:
S = iup.GetSummary(TrainData)
S

In [ ]:
S = iup.GetSummary(DevData)
S

## **Step 3.1 : (STEP NOT NEEDED)Select only 3 classes for extreme case model**
### Since now we are training "Filewise" and NOT "Valence" wise

In [ ]:
#from numpy import mean
#from numpy import std

# define dataset
#TrainDataSet = pd.DataFrame([])
#DevDataSet = pd.DataFrame([])
#for Valence in [1.5000,3.0000,4.5000]:
#  filte = (TrainData["V"] == Valence)
#  VCurr = TrainData[filte]
#  TrainDataSet = pd.concat([TrainDataSet,VCurr])

#  filte = (DevData["V"] == Valence)
#  VCurr = DevData[filte]
#  DevDataSet = pd.concat([DevDataSet,VCurr])
TrainDataSet = TrainData #For compatibility with forthcoming code
DevDataSet = DevData

In [ ]:
S = iup.GetSummary(TrainDataSet)
S

In [ ]:
S = iup.GetSummary(DevDataSet)
S

## **Step 3.2 : Map to -1,0,1**

In [ ]:
TrainNT = 2.0
TrainPT = 3.0
#Mapping = {1.5:-1, 3.0:0, 4.5:1}
TrainDataSet['V'] = TrainDataSet['V'].apply(lambda x : -1 if x < TrainNT else 1 if x > TrainPT else 0)
DevDataSet['V'] = DevDataSet['V'].apply(lambda x : -1 if x < TrainNT else 1 if x > TrainPT else 0)

In [ ]:
iup.GetSummary(TrainDataSet)

In [ ]:
iup.GetSummary(DevDataSet)

## **Step 4 : Model Training on Train Set and observing the thresholds**

In [ ]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
RandomForestRegressor()
model = BaggingRegressor(base_estimator = RandomForestRegressor(n_estimators = 3, n_jobs = -1, bootstrap=False,min_samples_leaf=5),n_estimators = 3, verbose=1, n_jobs = -1,bootstrap=True)
model.fit(TrainDataSet['Feature'].tolist(),TrainDataSet['V'].tolist())

In [ ]:
TrainDataResults = pd.DataFrame([])
TrainDataResults['SampleName'] = TrainDataSet['SampleName']
TrainDataResults['Actual'] = TrainDataSet['V']

In [ ]:
TrainDataResults['Predicted'] = model.predict(TrainDataSet['Feature'].tolist())
TrainDataResults = TrainDataResults.sort_values('Actual')

## **Step 6.1 : Statistical Guess of the Thresholds**

Here, for initial guess,

NT = mean(predictions for input of 1) + Std Dev(predictions for input of 1)

PT = mean(predictions for input of 5) - Std Dev(predictions for input of 5)

In [ ]:
import numpy as np
print(np.mean(np.array(TrainDataResults['Predicted'][0:3000].tolist())) + np.std(np.array(TrainDataResults['Predicted'][0:3000].tolist())))
print(np.mean(np.array(TrainDataResults['Predicted'][6000:9000].tolist())) - np.std(np.array(TrainDataResults['Predicted'][6000:9000].tolist())))

## **Step 6.2 : Adjustment of the Thresholds**

In [ ]:
PT = 0.15
NT = -0.24
TrainDataResults['Thresholded'] = TrainDataResults['Predicted'].apply(lambda x : -1.0 if x < NT else 1.0 if x > PT else 0.0)

In [ ]:
iup.plotResults(TrainDataResults,NT,PT,'Train','Experiments/BGRFTrain')

In [ ]:
iup.GetCMandMetrics(TrainDataResults,'Train','Experiments/TrainCM')

## **Step 7 : Run Model on Validation and apply Thresholds on them**

In [ ]:
DevDataResults = pd.DataFrame([])
DevDataResults['SampleName'] = DevDataSet['SampleName']
DevDataResults['Actual'] = DevDataSet['V']

In [ ]:
DevDataResults['Predicted'] = model.predict(DevDataSet['Feature'].tolist())
DevDataResults = DevDataResults.sort_values('Actual')

In [ ]:
iup.plotResults(DevDataResults,NT,PT,'Development','Experiments/BGRFDev')

In [ ]:
DevDataResults['Thresholded'] = DevDataResults['Predicted'].apply(lambda x : -1.0 if x < NT else 1.0 if x > PT else 0.0)

In [ ]:
iup.GetCMandMetrics(DevDataResults,'Development','Experiments/BGRFDevCM')

## **Step 8 : Save the Experiment Log**

## **Step 9 : Obtain Predictions on File Level**

In [ ]:
FilePredictions = pd.DataFrame([])
for Filename in tqdm(Files_Next500):
    TempTable = DevDataResults
    Temp = TempTable.loc[TempTable['SampleName'].apply(lambda x : x.startswith(Filename))]
    Predictions = Temp['Thresholded'].to_numpy()
    if not(Predictions.shape[0] == 0):
        TempDict = {'TURN_NAME' : Filename, 'Predictions' : Predictions, 'Actual' : Temp['Actual'].iloc[0]}
        FilePredictions = FilePredictions.append(TempDict,ignore_index=True)

In [ ]:
FilePredictions

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats
plt.bar(height = FilePredictions['Predictions'].iloc[128])
print('Standard Deviation : ',np.std(FilePredictions['Predictions'].iloc[128]))
print('Mean : ',np.mean(FilePredictions['Predictions'].iloc[128]))
print('Kurtosis : ',stats.kurtosis(FilePredictions['Predictions'].iloc[128]))
print('Skew : ',stats.skew(FilePredictions['Predictions'].iloc[128]))
print('ChiSquare : ',stats.chisquare(FilePredictions['Predictions'].iloc[128]))

In [ ]:
stats.chisquare(FilePredictions['Predictions'].iloc[128])

In [ ]:
e = FilePredictions['Predictions'].iloc[128]